# Fine-Tune a Generative AI Model for Dialogue Summarization

Di minggu ini memungkinkan Anda untuk mencoba fine-tuning menggunakan PEFT dengan LoRA sendiri dengan meningkatkan kemampuan ringkasan model Flan-T5.Di Lab 2, Anda akan berlatih dengan ***full fine-tuning*** dan ***Fine-Tuning Parameter-Efficient* (PEFT)**, juga disebut **PEFT dengan instruksi prompt**. Anda akan ***fine-tune* model Flan-T5 lebih lanjut dengan prompt khusus Anda sendiri untuk tugas ringkasan Anda yang spesifik**. Mari langsung beralih ke buku catatan. Di Lab 2, kita akan benar-benar *fine-tune* model. Di Lab 1, kita melakukan *zero-shot inference*, pembelajaran dalam konteks. Sekarang kita benar-benar akan **memodifikasi bobot model bahasa kita, khusus untuk tugas ringkasan kita dan spesifik untuk dataset kita**.

Mari kita lakukan instalasi pip ini. Sementara instalasi pip sedang berlangsung, biarkan saya menjelaskan tentang torch dan torchdata sama seperti Lab 1 di mana kita akan menggunakan PyTorch, kemudian kita memasang **library torchdata untuk membantu dengan pemuatan data PyTorch**. Ada juga **library bernama evaluates, dan ini yang akan kita gunakan dengan skor rouge untuk menghitung rouge**. Anda mempelajari tentang **rouge dalam pelajaran sebagai cara untuk mengukur seberapa baik ringkasan menggambarkan apa yang ada dalam percakapan asli atau teks asli**. Sekarang, dua library ini, LoRA dan PEFT, Anda mendengarnya sedikit dalam pelajaran. Ini yang akan kita gunakan untuk melakukan fine-tuning yang efisien dalam parameter (PEFT).

In [1]:
%pip install torch
%pip install torchdata
%pip install transformers
%pip install datasets
%pip install -U datasets 
%pip install evaluate
%pip install rouge_score
%pip install loralib
%pip install peft

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 1. Set up Kernel, Load Required Dependencies, Dataset and LLM

### 1.1 Set up Kernel and Required Dependencies

Kita memiliki **AutoModelForSeq2Seq**. Ini yang akan memberi kita akses ke Flan-T5 melalui perpustakaan python transformers, tokenizer, kita menggunakan generation config di lab sebelumnya.

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

C:\Users\247\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\247\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Mari muat dataset seperti yang kita lakukan di lab pertama.

### 1.2 Load Dataset and LLM

In [2]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Mari **muat model** seperti yang kita lakukan di lab pertama dan tokenizer, dan ini disebut **model asli** dan ini akan **berguna nanti saat kita membandingkan semua strategi fine-tuning dengan model asli yang tidak disesuaikan.**

In [3]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

C:\Users\247\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\247\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Berikut adalah fungsi kenyamanan yang **mencetak semua parameter yang ada dalam model dan khususnya parameter yang dapat dilatih**. Ini akan menjadi berguna ketika kita **memperkenalkan versi model PEFT yang tidak melatih semua parameter**. 

In [4]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model params: {trainable_model_params}\nall model params: {all_model_params}"

print(print_number_of_trainable_model_parameters(original_model))

trainable model params: 247577856
all model params: 247577856


Di sini kita melihat ada **sekitar 250 juta parameter yang dilatih ketika kita melakukan *full fine-tuning***, yang merupakan bagian pertama dari lab ini di mana kita melakukan *full fine-tuning*. Bagian kedua dari lab ini akan menjadi di mana kita melakukan **fine-tuning efisien parameter (PEFT) khusus dengan LoRA**, di mana kita **hanya akan melatih jumlah yang sangat kecil**. Jadi ingatlah, ini adalah jenis kode yang agak berantakan tapi cukup berguna untuk perbandingan.

### 1.3 Test and Model with Zero Shot Inferences

Sama seperti yang kita lakukan di lab pertama, kita akan menunjukkan contoh input. Kita akan menunjukkan baseline manusia. Kita akan melakukan ***zero-shot***. Ini bukan one shot, bukan few shot, kita lewati itu, itu adalah Lab 1. Di sini, kita mencoba mencapai titik di mana **satu panggilan sederhana ke model kita bisa memberikan ringkasan yang layak tanpa harus melewati one-shot dan few shot contoh**, itulah tujuannya.

In [5]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens = True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

## 2. Perform Full Fine-Tuning

### 2.1 Preprocess the Dialog Summary Dataset

Cara pertama yang akan kita lakukan adalah kita akan **melakukan full fine-tuning**. Berikut adalah **fungsi kenyamanan yang dapat menokenisasi dan membungkus dataset kita dalam sebuah prompt**. Seperti yang kita lihat di lab pertama di mana kita memiliki prompt yang mengatakan ringkaslah percakapan berikut, dan kemudian kita sebenarnya akan memberikan dialog tersebut, dan kemudian kita akan mengakhiri prompt dengan ringkasan itu titik dua. **Fungsi ini akan memungkinkan kita memetakan semua elemen dataset kita dan mengubahnya menjadi prompt dengan instruksi**. Itulah yang akan kita lakukan di sini, yaitu **full fine-tuning dengan prompt instruksi.**

In [6]:
def tokenize_function(example):
    start_prompt = 'summarize the following conversation. \n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary'])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Di sini, kita hanya akan mengambil sampel hanya untuk menjaga kebutuhan sumber daya untuk lab tertentu ini, mempercepat sedikit. Mari kita lihat ukurannya.

Untuk menghemat waktu dalam lab, Anda akan **mengambil sampel acak dari dataset**:

In [7]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Periksa bentuk dari ketiga bagian dataset ini:

In [8]:
print(f"shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


Di sini kita memiliki sekitar **125 contoh pelatihan**. Kita akan menggunakan **5 untuk validasi**. Kita akan menggunakan **15 untuk benar-benar melakukan uji coba holdout nanti saat kita membandingkan**. Kita akan **fine-tune dengan pelatihan dan kita akan memvalidasi dengan validasi**. Kemudian setelah semuanya selesai, kita akan menggunakan **15 contoh uji untuk kemudian membandingkan berbagai strategi untuk fine-tuning dengan instruksi**.

### 2.2 Fine-Tune the Model with Preprocessed Dataset

Di sini kita melihat **argumen pelatihan** dan kita melihat beberapa **default di sini untuk tingkat pembelajaran**. Kita **melihat beberapa nilai yang cukup rendah untuk langkah maksimum dan jumlah epoch**. Itu karena kita ingin mencoba meminimalkan jumlah komputasi yang diperlukan untuk lab ini. Jika Anda memiliki lebih banyak waktu, Anda tentu bisa mengubah nilai-nilai ini dan menaikkannya mungkin menjadi lima epoch, mungkin langkah maksimum 100. Sesaat lagi, saya akan menunjukkan bagaimana kita sebenarnya bekerja di sekitarnya. Kami telah melatih secara offline model yang jauh lebih besar dengan langkah maksimum yang lebih tinggi dan epoch pelatihan yang lebih tinggi dan sebentar lagi, kami akan benar-benar menariknya dan kemudian melanjutkan dari sana. Tetapi inilah tampilan kode. Inilah dataset pelatihan, inilah dataset evaluasi validasi, inilah tempat kita memanggil pelatihan. Sebenarnya biarkan saya hanya melakukan Shift Enter, mulailah ini. Ini akan memakan beberapa menit, bahkan dengan langkah maksimum yang rendah dan jumlah epoch yang rendah, ini masih membutuhkan beberapa menit untuk dijalankan.

Sekarang gunakan Trainer class bawaan dari Hugging Face. Berikan dataset yang sudah diproses dengan referensi ke model asli. Parameter pelatihan lainnya ditemukan secara eksperimental dan tidak perlu menjelaskan detail tentang itu saat ini.

In [9]:
import warnings
from accelerate import DataLoaderConfiguration, Accelerator

# Menghilangkan peringatan FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

# Konfigurasi DataLoaderConfiguration
dataloader_config = DataLoaderConfiguration(
    dispatch_batches=None, 
    split_batches=False, 
    even_batches=True, 
    use_seedable_sampler=True
)

# Membuat objek Accelerator dengan konfigurasi DataLoaderConfiguration
accelerator = Accelerator(dataloader_config=dataloader_config)

# Sekarang Anda dapat menggunakan objek accelerator dengan benar


In [10]:
output_dir = f'./dialogue_summary_training'

training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate = 1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'], # training dataset 
    eval_dataset=tokenized_datasets['validation']) # evaluation dataset

Proses training dimulai...

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(output_dir)

Buatlah sebuah contoh dari **AutoModelForSeq2SeqLM class** untuk instruct model:

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./dialogue_summary_training", torch_dtype=torch.bfloat16)

### 2.3 Evaluate the Model Qualitatively (Human Evaluation)

Seperti halnya dengan banyak aplikasi AI, pendekatan kualitatif di mana Anda bertanya pada diri sendiri, "apakah model saya berperilaku sesuai yang diharapkan?" biasanya merupakan titik awal yang baik. Pada contoh di bawah ini (sama dengan yang kita mulai di notebook ini), Anda dapat melihat **bagaimana model yang fine-tuned dapat membuat ringkasan yang masuk akal dari dialog dibandingkan dengan ketidakmampuan asli untuk memahami apa yang diminta dari model.**

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation:

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_outputs = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_outputs = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(f'dialogue:\n{dialogue}\n')
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}\n')
print(f'ORIGINAL MODEL:\n{original_model_text_outputs}\n')
print(f'INSTRUCT MODEL:\n{instruct_model_text_outputs}\n')

---------------------------------------------------------------------------------------------------
**BASELINE HUMAN SUMMARY:**

#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

**ORIGINAL MODEL:**

#Person1#: You'd like to upgrade your computer. #Person2: You'd like to upgrade your computer.

**INSTRUCT MODEL:**

#Person1# suggests #Person2# upgrading #Person2#'s system, hardware, and CD-ROM drive. #Person2# thinks it's great.

### 2.4 Evaluate the Model Quantitatively (with ROUGE Metric)

**Metrik ROUGE membantu mengkuantifikasi validitas ringkasan yang dihasilkan oleh model**. Ini membandingkan ringkasan dengan ringkasan "baseline" yang biasanya dibuat oleh manusia. Meskipun tidak sempurna, **metrik ini menunjukkan peningkatan keseluruhan dalam efektivitas ringkasan yang telah kita capai dengan fine-tuning.**

In [ ]:
rouge = evaluate.load('rouge')

Hasilkan output untuk sampel dataset uji (hanya 10 percakapan dan ringkasan untuk menghemat waktu), dan simpan hasilnya.

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)
    
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

Khususnya, mari kita muat ROUGE dan kita akan melihat, saya pikir kita hanya akan melakukan mungkin 10 pertama di sini, dan mari kita bandingkan. **Mari ambil 10 pertama dari dataset uji kami**. Kami akan menjalankannya melalui percakapan ini, melalui **model Flan-T5 asli serta model fine-tuning instruksi yang kita latih di atas**. Di sini, tentu saja, **kami akan membungkusnya dalam prompt yang mirip dengan apa yang kami gunakan untuk pelatihan**. Kemudian mari kita lihat bagaimana hasilnya. Ini adalah cara kualitatif untuk melihat keduanya secara berdampingan.

Evaluasi model dengan menghitung metrik ROUGE. Perhatikan peningkatan hasilnya.

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

**ORIGINAL MODEL:**

{'rouge1': 0.24223171760013867, 'rouge2': 0.10614243734192583, 'rougeL': 0.21380459196706333, 'rougeLsum': 0.21740921541379205}

**INSTRUCT MODEL:**

{'rouge1': 0.41026607717457186, 'rouge2': 0.17840645241958838, 'rougeL': 0.2977022096267017, 'rougeLsum': 0.2987374187518165}

Mari kita bandingkan metrik ROUGE untuk model Flan-T5 asli dan model fine-tune instruksi yang telah kita tuning di atas. Di sini kita melihat bahwa **skor model fine-tune instruksi jauh lebih tinggi pada metrik evaluasi ROUGE daripada model Flan-T5 asli**. Ini menunjukkan bahwa dengan sedikit fine-tuning menggunakan dataset kami dan prompt yang spesifik, kami sebenarnya dapat meningkatkan metrik ROUGE.

File data/dialogue-summary-training-result.csv berisi daftar yang sudah diisi sebelumnya dari semua hasil model yang dapat Anda gunakan untuk mengevaluasi pada sebagian data yang lebih besar. Mari lakukan itu untuk setiap model.

In [ ]:
results = pd.read_csv("data/dialogue-summary-training-results.csv")

human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

**ORIGINAL MODEL:**

{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}

**INSTRUCT MODEL:**

{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}

Satu hal lain yang kami lakukan secara offline adalah kami melakukan ini dengan dataset uji yang jauh lebih besar dan lebih lama. Ini bukan hanya 10 atau 15 contoh, sebenarnya ini adalah dataset penuh, dan mari kita lihat. Itu adalah apa yang ada dalam file ini. File CSV yang disertakan dalam direktori data dengan lab ini. **Di sini kita lihat dengan dataset yang jauh lebih besar, skornya masih cukup mirip, di mana kita mendekati dua kali lipat,** meskipun tidak sepenuhnya dua kali lipat dalam beberapa kasus, **tetapi peningkatan cukup signifikan dari Flan-T5 asli.** 

**Hasilnya menunjukkan peningkatan yang signifikan** dalam semua metrik ROUGE:

In [ ]:
print("Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Di sini kita lihat persentase peningkatan secara khusus. Jika kita benar-benar melakukan perhitungan, kita lihat rouge1 18% lebih tinggi, rouge2 10%, rougeL 13, rougeLsum 13,7 juga. 

## 3. Perform Parameter Efficient Fine-Tuning (PEFT)

Sekarang mari masuk ke fine-tuning parameter yang efisien (PEFT). Ini membuat perbedaan besar, terutama ketika Anda dibatasi oleh seberapa banyak sumber daya komputasi yang Anda miliki, Anda **dapat menurunkan jejaknya baik memori, disk, GPU, CPU, semua sumber daya dapat dikurangi hanya dengan memperkenalkan PEFT ke dalam proses fine-tuning Anda**.

### 3.1. Setup the PEFT /LoRA model for Fine-Tuning

Anda perlu menyiapkan model PEFT/LoRA untuk fine-tuning dengan lapisan/parameter adapter baru. **Menggunakan PEFT/LoRA, Anda membekukan LLM yang mendasari dan hanya melatih adapter.** Lihat konfigurasi **LoRA** di bawah ini. Perhatikan **hyperparameter rank (r)**, yang mendefinisikan **rank/dimensi adapter yang akan dilatih.**

Dalam pelajaran Anda belajar tentang **LoRA**, Anda belajar tentang **peringkat (rank)**. Di sini kita akan memilih peringkat 32, yang sebenarnya relatif tinggi. Tapi kita baru saja mulai dengan itu. Di sini adalah **SEQ_2_SEQ_LM**, ini adalah **FLAN-T5**. Dengan hanya beberapa baris kode ekstra di sini untuk **mengonfigurasi fine-tuning LoRA** kami.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

**Tambahkan lapisan/parameter adapter LoRA ke LLM asli untuk dilatih.**

In [ ]:
peft_model = get_peft_model(original_model,
                           lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944

all model parameters: 251116800

percentage of trainable model parameters: 1.41%

Kemudian di sini kita hanya akan **melatih 1,4 persen dari parameter model yang dapat dilatih.**

Dalam banyak kasus, Anda dapat fine-tuning model yang sangat besar pada satu GPU. Berikut adalah beberapa argumen pelatihan tersebut. Ini kembali ke argumen pelatihan asli dari hugging face, kecuali **daripada hanya menggunakan model biasa, kami sebenarnya menggunakan model PEFT.** Di sini ini adalah fungsi kenyamanan yang ditawarkan oleh perpustakaan PEFT dan **kami memberikan model asli, yang merupakan FLAN-T5.** Kami memberikan **konfigurasi LoRA** yang kita tentukan di atas dengan Peringkat 32. Kami katakan berikan saya versi PEFT dari model itu. Itu keluar sebagai 1,4 persen. 

### 3.2 Train PEFT Adapter

Sekarang kita lakukan argumen pelatihan. Sekali lagi, jumlah langkah yang kecil, jumlah epoch yang kecil di sini. Kami memiliki versi yang dilatih secara offline. Itu sedikit lebih baik daripada yang ada di lab ini secara khusus, dan itulah yang akan kita unduh di sini dalam beberapa saat. 

**Definisikan argumen pelatihan dan buat instance Trainer**

In [ ]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_largs = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, #higher learning rate than full fine-tuning.
    num_train_epochs=1, 
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

Sekarang semuanya sudah **siap untuk melatih adapter PEFT dan menyimpan model.**

In [ ]:
peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Mari kita lakukan itu. Inilah **model lain yang disimpan dalam penyimpanan cloud S3.**

In [ ]:
('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

Pelatihan tersebut dilakukan pada subset data. Untuk memuat model PEFT yang sudah dilatih sepenuhnya, baca checkpoint model PEFT dari S3.

In [ ]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/ ./peft-dialogue-summary-checkpoint-from-s3/ 

**Pastikan ukuran model ini jauh lebih kecil dibandingkan LLM asli.**

In [ ]:
!ls -al ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin

Sekarang kita melihat ini hanya 14 megabyte. Ini disebut sebagai **adapter PEFT atau pengadopsi LoRA.** Ini digabungkan atau digabungkan dengan LLM asli. Ketika Anda benar-benar melayani model ini, yang akan kita dengar dalam waktu dekat, Anda harus **mengambil LLM asli dan kemudian menggabungkan adapter PEFT LoRA ini.** **Ini jauh lebih kecil dan Anda dapat menggunakan kembali LLM dasar yang sama dan menukar adapter PEFT yang berbeda jika diperlukan.**

**Siapkan model ini dengan menambahkan adapter ke model FLAN-T5 asli.** Anda mengatur is_trainable=False karena rencana hanya melakukan inferensi dengan model PEFT ini. Jika Anda menyiapkan model untuk pelatihan lebih lanjut, Anda akan mengatur is_trainable=True.

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                      './peft-dialogue-summary-checkpoint-from-s3',
                                      torch_dtype=torch.bfloat16,
                                      is_trainable=False)

The number of trainable parameters will be 0 due to is_trainable=False setting:

Sekarang s**etelah kita memiliki adapter PEFT disalin dari S3, kita akan menggabungkannya dengan LLM asli, yaitu FLAN-T5, dan menggunakan itu untuk melakukan summarisasi.** Satu hal yang perlu ditekankan yang tidak sepenuhnya jelas adalah bahwa saat kita melakukan ini, saya sebenarnya dapat mengatur flag is_trainable menjadi false. **Dengan mengatur flag is_trainable menjadi false, kita memberi tahu PyTorch bahwa kita tidak tertarik untuk melatih model ini. Yang kami minati hanyalah langkah maju hanya untuk mendapatkan ringkasan.** Ini penting karena kita dapat memberi tahu PyTorch untuk tidak memuat bagian pembaruan dari operator-operator ini dan pada dasarnya meminimalkan jejak yang dibutuhkan untuk hanya melakukan inferensi dengan model ini. Ini adalah bendera yang cukup bagus. Ini sebenarnya baru saja diperkenalkan baru-baru ini ke dalam model PEFT pada saat lab ini. Saya ingin menunjukkannya di sini karena ini adalah pola yang ingin Anda coba temukan saat Anda melakukan pemodelan Anda sendiri. **Ketika Anda tahu bahwa Anda siap untuk mendeploy model untuk inferensi, biasanya ada cara di mana Anda dapat memberi petunjuk kepada kerangka kerja, seperti PyTorch bahwa Anda tidak akan melakukan pelatihan. Ini kemudian dapat lebih mengurangi sumber daya yang dibutuhkan untuk membuat prediksi ini.**

In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0

all model parameters: 251116800

percentage of trainable model parameters: 0.00%

Di sini, hanya untuk menekankannya, saya **mencetak jumlah parameter yang dapat dilatih.** Ingatlah bahwa pada titik ini **kita hanya berencana untuk melakukan inferensi**, dan mari kita lanjutkan. 0 % dari parameter yang dapat dilatih ini.

### 3.3. Evaluate the Model Qualitative (Human Evaluation)

Lakukan inferensi untuk contoh yang sama seperti pada bagian 1.3 dan 2.3, dengan model asli, *fully fine-tuned*, dan model PEFT.

Di sini, kita akan membuat beberapa contoh prompt dari dataset uji kami. Kita hanya akan memilih sesuatu secara acak di sini, pada dasarnya Indeks 200. Kita akan melihat model instruksi. 

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
**BASELINE HUMAN SUMMARY:**

#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

**ORIGINAL MODEL:**

#Pork1: Have you considered upgrading your system? #Person1: Yes, but I'd like to make some improvements. #Pork1: I'd like to make a painting program. #Person1: I'd like to make a flyer. #Pork2: I'd like to make banners. #Person1: I'd like to make a computer graphics program. #Person2: I'd like to make a computer graphics program. #Person1: I'd like to make a computer graphics program. #Person2: Is there anything else you'd like to do? #Person1: I'd like to make a computer graphics program. #Person2: Is there anything else you need? #Person1: I'd like to make a computer graphics program. #Person2: I'

**INSTRUCT MODEL:**

#Person1# suggests #Person2# upgrading #Person2#'s system, hardware, and CD-ROM drive. #Person2# thinks it's great.

**PEFT MODEL:**

#Person1# recommends adding a painting program to #Person2#'s software and upgrading hardware. #Person2# also wants to upgrade the hardware because it's outdated now.

Mengerti. Sebagian besar benar saya kira, model PEFT menemukan sedikit lebih banyak nuansa di sini. Tetapi sebenarnya, seperti yang akan kita lihat secara kualitatif ketika kita menjalankan metrik rouge. 

### 3.4. Evaluate the Model Quantitatively (with ROUGE Metric)

Lakukan inferensi untuk sampel dari data uji (hanya 10 dialog dan ringkasan untuk menghemat waktu).

Di sini kita akan **membandingkan baseline manusia dengan FLAN-T5 asli dengan full fine-tuned instruksi, dan kemudian dengan PEFT fine-tuned.**

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]
    
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

Sebagian besar, hanya sekilas di sini, sepertinya ini cukup mirip. 

 Tapi mari kita lihat **metrik rouge** dan lihat apa yang terjadi.

In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

**ORIGINAL MODEL:**

{'rouge1': 0.2127769756385947, 'rouge2': 0.07849999999999999, 'rougeL': 0.1803101433337705, 'rougeLsum': 0.1872151390166362}

**INSTRUCT MODEL:**

{'rouge1': 0.41026607717457186, 'rouge2': 0.17840645241958838, 'rougeL': 0.2977022096267017, 'rougeLsum': 0.2987374187518165}

**PEFT MODEL:**

{'rouge1': 0.3725351062275605, 'rouge2': 0.12138811933618107, 'rougeL': 0.27620639623170606, 'rougeLsum': 0.2758134870822362}

Di sini kita lihat **model fine-tuned instruksi adalah peningkatan yang cukup drastis dibandingkan dengan FLAN-T5 asli.** Kita melihat bahwa **model PEFT mengalami sedikit degradasi dari full fine-tuned.** Ini cukup dekat dalam beberapa kasus. Itu tidak terlalu buruk. Tetapi **kita menggunakan sumber daya yang jauh lebih sedikit selama fine-tuning, daripada yang akan kita miliki jika kita melakukan instruksi penuh.** Anda dapat membayangkan ini hanya beberapa ribu sampel, tetapi Anda dapat membayangkan dalam skala besar bagaimana ini benar-benar dapat menghemat banyak sumber daya komputasi dan waktu dengan menggunakan PEFT dengan melihat dataset yang lebih besar. 

Perhatikan bahwa **hasil model PEFT tidak terlalu buruk, sementara proses pelatihannya jauh lebih mudah!**

Anda sudah menghitung skor ROUGE pada seluruh dataset, setelah memuat hasil dari file data/dialogue-summary-training-results.csv. Sekarang, **muat nilai untuk model PEFT dan periksa kinerjanya dibandingkan dengan model lain.**

In [ ]:
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values
peft_model_summaries     = results['peft_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

**ORIGINAL MODEL:**

{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}

**INSTRUCT MODEL:**

{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}

**PEFT MODEL:**

{'rouge1': 0.40810631575616746, 'rouge2': 0.1633255794568712, 'rougeL': 0.32507074586565354, 'rougeLsum': 0.3248950182867091}

**Hasil menunjukkan peningkatan yang lebih rendah dibandingkan dengan pelatihan penuh, tetapi manfaat PEFT biasanya lebih besar daripada sedikit penurunan metrik kinerja.**

Hitung peningkatan PEFT dibandingkan dengan model asli:

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE

rouge1: 17.47%

rouge2: 8.73%

rougeL: 12.36%

rougeLsum: 12.34%

Di atas, saya hanya melihat mungkin 10, 15 contoh. Di sini kita lihat yang lebih besar. Sepertinya saya pikir saya memiliki ini **di sini rouge satu, PEFT kehilangan sekitar satu hingga mungkin 1,7 persen di seluruh metrik rouge ini.** Itu tidak buruk relatif terhadap penghematan yang Anda dapatkan saat menggunakan PEFT.

Hitung **peningkatan PEFT dibandingkan dengan model yang sepenuhnya disempurnakan**:

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL

rouge1: -1.35%

rouge2: -1.70%

rougeL: -1.34%

rougeLsum: -1.35%

Di sini, Anda melihat sedikit **penurunan persentase pada metrik ROUGE dibandingkan dengan model full fine-tuning.** Namun, **pelatihan membutuhkan jauh lebih sedikit sumber daya komputasi dan memori (seringkali hanya membutuhkan satu GPU).**